# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?  
目標均值編碼的效果仍然較佳.

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [4]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on=['Cabin'], how='left')
df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

C:\Users\allen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
822,"Reuchlin, Jonkheer. John George",male,19972,None,S,1014
830,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,2659,None,C,1014
828,"McCormack, Mr. Thomas Joseph",male,367228,None,Q,1014
827,"Mallet, Master. Andre",male,S.C./PARIS 2079,None,C,1014
826,"Lam, Mr. Len",male,1601,None,S,1014
825,"Flynn, Mr. John",male,368323,None,Q,1014
824,"Panula, Master. Urho Abraham",male,3101295,None,S,1014
821,"Lulic, Mr. Nikola",male,315098,None,S,1014
832,"Saad, Mr. Amin",male,2671,None,C,1014


In [5]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7856230275549181


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [6]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 100)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7811347807476239


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,75
1,286,0,816,106,0,95
2,523,0,914,185,3,75
3,422,0,65,70,3,97
4,22,1,649,185,3,75


In [7]:
# 加上 'Cabin' 欄位的均值編碼
CabinMean = pd.concat([df[:train_num], train_Y], axis=1)
mean_df = CabinMean.groupby(['Cabin'])['Survived'].mean().reset_index()
mean_df.columns = ['Cabin', 'Cabin_mean']
df = pd.merge(CabinMean, mean_df, on='Cabin', how='left')
df.sort_values(by=['Cabin_mean'], ascending=False).head(10)

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count,Survived,Cabin_mean
445,"Dodge, Master. Washington",male,33638,A34,S,3,1,1.0
690,"Dick, Mr. Albert Adrian",male,17474,B20,S,2,1,1.0
224,"Hoyt, Mr. Frederick Maxfield",male,19943,C93,S,2,1,1.0
393,"Newell, Miss. Marjorie",female,35273,D36,C,2,1,1.0
319,"Spedden, Mrs. Frederic Oakley (Margaretta Corn...",female,16966,E34,C,3,1,1.0
390,"Carter, Mr. William Ernest",male,113760,B96 B98,S,4,1,1.0
550,"Thayer, Mr. John Borland Jr",male,17421,C70,C,1,1,1.0
689,"Madill, Miss. Georgette Alexandra",female,24160,B5,S,2,1,1.0
781,"Dick, Mrs. Albert Adrian (Vera Gillespie)",female,17474,B20,S,2,1,1.0
609,"Shutes, Miss. Elizabeth W",female,PC 17582,C125,S,2,1,1.0


In [8]:
# 'Cabin'均值編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_mean'] = df['Cabin_mean']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.8316784565642724


,Name,Sex,Ticket,Cabin,Embarked,Cabin_mean
0,108,1,523,146,3,0.299854
1,190,0,596,81,0,1.000000
2,353,0,669,146,3,0.299854
3,272,0,49,55,3,0.500000
4,15,1,472,146,3,0.299854


In [9]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count']
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 100)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7833819717588598


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count,Cabin_Hash
0,108,1,523,146,3,1014,75
1,190,0,596,81,0,2,95
2,353,0,669,146,3,1014,75
3,272,0,49,55,3,2,97
4,15,1,472,146,3,1014,75
